# Chapter 14: Enums and Choosing the Right Data Structure

This notebook covers Python's `enum` module for defining named constants and provides guidance on choosing the right data structure for different situations.

## Key Concepts
- **Enum**: Named constants with values, preventing magic numbers/strings
- **auto()**: Automatic value generation
- **IntEnum, StrEnum**: Type-compatible enum variants
- **Flag, IntFlag**: Bitmask enums for combining options
- **Choosing structures**: When to use list, tuple, set, dict, and collections types
- **Performance characteristics**: Understanding O(1) vs O(n) tradeoffs

## Basic Enum: Named Constants with Values

Enums replace magic numbers and strings with meaningful names. Each member has a `name` (string) and a `value`.

In [ ]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

# Access by name and value
print(f"Color.RED = {Color.RED}")
print(f"Name: {Color.RED.name}")
print(f"Value: {Color.RED.value}")

# Lookup by value
color = Color(2)
print(f"\nColor(2) = {color}")
print(f"Color(2) is Color.GREEN: {color is Color.GREEN}")

# Lookup by name
color = Color["BLUE"]
print(f"Color['BLUE'] = {color}")

# Enum members are singletons -- identity comparison works
print(f"\nColor.RED is Color.RED: {Color.RED is Color.RED}")

# String enums work too
class Status(Enum):
    PENDING = "pending"
    ACTIVE = "active"
    CLOSED = "closed"

print(f"\nStatus.ACTIVE.value = {Status.ACTIVE.value!r}")

## auto() for Automatic Value Generation

`auto()` generates values automatically (starting from 1 by default). Use it when you care about the name but not the specific value.

In [ ]:
from enum import Enum, auto

class Direction(Enum):
    NORTH = auto()
    SOUTH = auto()
    EAST = auto()
    WEST = auto()

print("Direction members:")
for d in Direction:
    print(f"  {d.name} = {d.value}")

print(f"\nTotal directions: {len(Direction)}")

# auto() values start at 1 by default
assert Direction.NORTH.value == 1
assert Direction.WEST.value == 4

# You can customize auto() by overriding _generate_next_value_
class LowerCaseEnum(Enum):
    @staticmethod
    def _generate_next_value_(name: str, start: int, count: int, last_values: list) -> str:
        return name.lower()

    RUNNING = auto()
    STOPPED = auto()
    PAUSED = auto()

print(f"\nCustom auto values:")
for item in LowerCaseEnum:
    print(f"  {item.name} -> {item.value!r}")

## Enum Iteration and Membership Testing

Enums are iterable, and you can test membership using `in`. This makes them ideal for validation and exhaustive matching.

In [ ]:
from enum import Enum

class Priority(Enum):
    LOW = 1
    MEDIUM = 2
    HIGH = 3
    CRITICAL = 4

# Iteration preserves definition order
names = [p.name for p in Priority]
values = [p.value for p in Priority]
print(f"Names: {names}")
print(f"Values: {values}")

# Membership testing
print(f"\nPriority.HIGH in Priority: {Priority.HIGH in Priority}")

# Practical example: validate and dispatch
def handle_ticket(priority: Priority) -> str:
    """Route tickets based on priority."""
    match priority:
        case Priority.CRITICAL:
            return "Page on-call engineer immediately"
        case Priority.HIGH:
            return "Assign to senior engineer"
        case Priority.MEDIUM:
            return "Add to sprint backlog"
        case Priority.LOW:
            return "Add to backlog"

for p in Priority:
    print(f"  {p.name}: {handle_ticket(p)}")

## IntEnum and StrEnum: Type-Compatible Enums

`IntEnum` members behave like integers and `StrEnum` members behave like strings. They can be used directly in arithmetic or string operations. Use with caution -- this breaks the type safety that regular `Enum` provides.

In [ ]:
from enum import IntEnum, StrEnum

# IntEnum: members are also integers
class HttpStatus(IntEnum):
    OK = 200
    NOT_FOUND = 404
    SERVER_ERROR = 500

# Can be used in integer operations
print(f"HttpStatus.OK == 200: {HttpStatus.OK == 200}")
print(f"HttpStatus.NOT_FOUND > 300: {HttpStatus.NOT_FOUND > 300}")
print(f"HttpStatus.OK + 1 = {HttpStatus.OK + 1}")

# Works where integers are expected
status_code: int = HttpStatus.OK
print(f"\nUsed as int: {status_code}")

# StrEnum: members are also strings (Python 3.11+)
class MediaType(StrEnum):
    JSON = "application/json"
    HTML = "text/html"
    PLAIN = "text/plain"

# Can be used in string operations
print(f"\nMediaType.JSON == 'application/json': {MediaType.JSON == 'application/json'}")
print(f"Content-Type: {MediaType.JSON}")
print(f"Starts with 'text': {MediaType.HTML.startswith('text')}")

# Use in f-strings directly
header = f"Content-Type: {MediaType.JSON}"
print(f"Header: {header}")

## Flag and IntFlag: Bitmask Operations

`Flag` enables combining enum members using bitwise operators (`|`, `&`, `~`). This is useful for permission systems, feature flags, and configuration options.

In [ ]:
from enum import Flag, auto, IntFlag

class Permission(Flag):
    READ = auto()     # 1
    WRITE = auto()    # 2
    EXECUTE = auto()  # 4
    DELETE = auto()   # 8

    # Convenient combined permissions
    READ_WRITE = READ | WRITE
    ALL = READ | WRITE | EXECUTE | DELETE

# Combine permissions with |
user_perms = Permission.READ | Permission.WRITE
print(f"User permissions: {user_perms}")
print(f"Value: {user_perms.value}")

# Test permissions with &
print(f"\nHas READ: {bool(user_perms & Permission.READ)}")
print(f"Has EXECUTE: {bool(user_perms & Permission.EXECUTE)}")
print(f"Has READ_WRITE: {bool(user_perms & Permission.READ_WRITE)}")

# Check membership with 'in'
print(f"\nREAD in user_perms: {Permission.READ in user_perms}")
print(f"DELETE in user_perms: {Permission.DELETE in user_perms}")

# Iterate over individual flags in a combined value
admin_perms = Permission.ALL
print(f"\nAdmin permissions: {admin_perms}")
print(f"Individual flags:")
for perm in Permission:
    if perm in admin_perms and perm.name not in ("READ_WRITE", "ALL"):
        print(f"  - {perm.name}")

## Choosing the Right Data Structure

Python offers many built-in and collections-based data structures. Choosing the right one depends on your access patterns, mutability needs, and performance requirements.

In [ ]:
# list vs tuple vs set vs dict -- when to use each

# LIST: ordered, mutable, allows duplicates
# Use for: sequences that change, maintaining order
tasks = ["write code", "test code", "deploy", "write code"]
tasks.append("monitor")
print(f"List (ordered, duplicates OK): {tasks}")

# TUPLE: ordered, immutable, allows duplicates
# Use for: fixed records, dict keys, return values
rgb = (255, 128, 0)
print(f"Tuple (fixed record): {rgb}")

# SET: unordered, mutable, unique elements only
# Use for: membership testing, deduplication
unique_tags = {"python", "data", "python", "web"}
print(f"Set (unique, unordered): {unique_tags}")

# DICT: key-value pairs, ordered (3.7+), mutable
# Use for: lookups by key, mappings, caches
user = {"name": "Alice", "age": 30, "role": "engineer"}
print(f"Dict (key-value mapping): {user}")

# FROZENSET: immutable set
# Use for: set operations on immutable data, dict keys
immutable_tags = frozenset(["python", "web"])
print(f"Frozenset (immutable set): {immutable_tags}")

## Performance Characteristics: O(1) vs O(n)

Understanding time complexity helps you choose the right structure. The difference between O(1) and O(n) becomes dramatic with large datasets.

In [ ]:
import time

# Compare lookup performance: list vs set vs dict
n = 100_000
data_list = list(range(n))
data_set = set(range(n))
data_dict = {i: True for i in range(n)}

target = n - 1  # Worst case for list (last element)

# List lookup: O(n) - must scan sequentially
start = time.perf_counter()
for _ in range(1000):
    _ = target in data_list
list_time = time.perf_counter() - start

# Set lookup: O(1) - hash-based
start = time.perf_counter()
for _ in range(1000):
    _ = target in data_set
set_time = time.perf_counter() - start

# Dict lookup: O(1) - hash-based
start = time.perf_counter()
for _ in range(1000):
    _ = target in data_dict
dict_time = time.perf_counter() - start

print(f"Lookup {target:,} in {n:,} items (1000 iterations):")
print(f"  list: {list_time:.4f}s  (O(n) - linear scan)")
print(f"  set:  {set_time:.4f}s  (O(1) - hash lookup)")
print(f"  dict: {dict_time:.4f}s  (O(1) - hash lookup)")
print(f"\nSet is ~{list_time / set_time:.0f}x faster than list for this lookup")

# Common operations and their complexity
print("\n--- Time Complexity Reference ---")
print(f"{'Operation':<30} {'list':>8} {'set':>8} {'dict':>8}")
print("-" * 58)
print(f"{'Lookup (in)':<30} {'O(n)':>8} {'O(1)':>8} {'O(1)':>8}")
print(f"{'Append/Add':<30} {'O(1)':>8} {'O(1)':>8} {'O(1)':>8}")
print(f"{'Insert at front':<30} {'O(n)':>8} {'N/A':>8} {'N/A':>8}")
print(f"{'Delete by value':<30} {'O(n)':>8} {'O(1)':>8} {'O(1)':>8}")
print(f"{'Index access':<30} {'O(1)':>8} {'N/A':>8} {'O(1)':>8}")
print(f"{'Iteration':<30} {'O(n)':>8} {'O(n)':>8} {'O(n)':>8}")

## When to Use Each Collections Type

Here is a decision guide for choosing between the standard library and `collections` types.

In [ ]:
from collections import Counter, defaultdict, deque, ChainMap
from dataclasses import dataclass
from typing import NamedTuple

# Scenario 1: Counting occurrences -> Counter
votes = ["alice", "bob", "alice", "charlie", "alice", "bob"]
results = Counter(votes)
winner = results.most_common(1)[0]
print(f"Election results: {results}")
print(f"Winner: {winner[0]} with {winner[1]} votes")

# Scenario 2: Grouping items -> defaultdict(list)
students = [("Math", "Alice"), ("Science", "Bob"), ("Math", "Charlie")]
by_subject: defaultdict[str, list[str]] = defaultdict(list)
for subject, student in students:
    by_subject[subject].append(student)
print(f"\nStudents by subject: {dict(by_subject)}")

# Scenario 3: Recent history / sliding window -> deque(maxlen=N)
recent_searches: deque[str] = deque(maxlen=3)
for query in ["python", "dataclass", "enum", "collections", "typing"]:
    recent_searches.append(query)
print(f"Recent searches: {list(recent_searches)}")

# Scenario 4: Immutable record -> NamedTuple or frozen dataclass
class APIResponse(NamedTuple):
    status: int
    body: str

resp = APIResponse(200, '{"ok": true}')
print(f"\nAPI response: {resp}")

# Scenario 5: Mutable record with validation -> dataclass
@dataclass
class ShoppingCart:
    items: list[str]
    total: float = 0.0

cart = ShoppingCart(items=["book"])
cart.items.append("pen")
cart.total = 15.99
print(f"Cart: {cart}")

## Practical Example: Building a Task Manager

Let's combine enums, dataclasses, and collections to build a small task management system.

In [ ]:
from collections import Counter, defaultdict
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import Iterator

class TaskStatus(Enum):
    TODO = auto()
    IN_PROGRESS = auto()
    DONE = auto()

class TaskPriority(Enum):
    LOW = 1
    MEDIUM = 2
    HIGH = 3

@dataclass
class Task:
    title: str
    assignee: str
    priority: TaskPriority = TaskPriority.MEDIUM
    status: TaskStatus = TaskStatus.TODO
    tags: list[str] = field(default_factory=list)

@dataclass
class TaskManager:
    tasks: list[Task] = field(default_factory=list)

    def add(self, task: Task) -> None:
        self.tasks.append(task)

    def by_status(self) -> dict[TaskStatus, list[Task]]:
        grouped: defaultdict[TaskStatus, list[Task]] = defaultdict(list)
        for task in self.tasks:
            grouped[task.status].append(task)
        return dict(grouped)

    def status_summary(self) -> Counter:
        return Counter(task.status.name for task in self.tasks)

    def high_priority(self) -> Iterator[Task]:
        return (t for t in self.tasks if t.priority == TaskPriority.HIGH)

# Build a task board
mgr = TaskManager()
mgr.add(Task("Design API", "Alice", TaskPriority.HIGH, tags=["backend"]))
mgr.add(Task("Write tests", "Bob", TaskPriority.MEDIUM, TaskStatus.IN_PROGRESS))
mgr.add(Task("Fix bug #42", "Alice", TaskPriority.HIGH, TaskStatus.IN_PROGRESS))
mgr.add(Task("Update docs", "Charlie", TaskPriority.LOW, TaskStatus.DONE))
mgr.add(Task("Deploy v2", "Alice", TaskPriority.HIGH, tags=["devops"]))

# Status summary using Counter
print("Status summary:")
for status, count in mgr.status_summary().most_common():
    print(f"  {status}: {count}")

# High-priority tasks
print("\nHigh priority tasks:")
for task in mgr.high_priority():
    print(f"  [{task.status.name}] {task.title} -> {task.assignee}")

# Grouped by status
print("\nTasks by status:")
for status, tasks in mgr.by_status().items():
    print(f"  {status.name}:")
    for t in tasks:
        print(f"    - {t.title} ({t.priority.name})")

## Summary

### Enum Key Takeaways
- Use **`Enum`** to replace magic numbers and strings with meaningful names
- Use **`auto()`** when you do not care about specific values
- Use **`IntEnum`/`StrEnum`** only when you need type compatibility with int/str
- Use **`Flag`** for combinable bitmask options (permissions, features)
- Enums are iterable, hashable, and work with `match`/`case`

### Data Structure Decision Guide

| Need | Best Choice |
|------|------------|
| Ordered, mutable sequence | `list` |
| Fixed record / dict key | `tuple` or `NamedTuple` |
| Fast membership testing | `set` or `frozenset` |
| Key-value mapping | `dict` |
| Counting elements | `Counter` |
| Auto-default for missing keys | `defaultdict` |
| Fast append/pop both ends | `deque` |
| Layered configuration | `ChainMap` |
| Structured mutable record | `@dataclass` |
| Structured immutable record | `@dataclass(frozen=True)` or `NamedTuple` |
| Named constants | `Enum` |
| Combinable flags | `Flag` |

### Performance Rule of Thumb
- Need frequent **lookups**? Use `set` or `dict` (O(1)) over `list` (O(n))
- Need frequent **insertion at both ends**? Use `deque` (O(1)) over `list` (O(n) for front)
- Need **memory efficiency** for many instances? Use `slots=True` or `NamedTuple`